In [1]:
import pandas as pd
from src.spreadsheet import SpreadSheet
from src.classifier import *

/Users/rishichandra/opt/anaconda3/envs/fingal/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Constants for the spreadsheet.

In [2]:
SPREADSHEET_ID = "1NONAyZN-DU7VyRR4ystC8rDZ0aBQsEU65kTJiesV-c0"

col_mappings = {
    "month": "I",
    "date": "K",
    "cost": "L",
    "description": "M",
    "category": "N",
}

Change this path for each update:

In [3]:
TRANSACTIONS_PATH = "data/new-data.csv"

Load and preprocess current spreadsheet and new transactions:

In [4]:
sheet = SpreadSheet(SPREADSHEET_ID)
old_data = load_and_process_spreadsheet(sheet, col_mappings)
new_data = load_and_process_transactions(TRANSACTIONS_PATH)
display(new_data)

,Date,Name,Amount
0,2024-10-17,NVIDIA CAFE SANTA CLARA CA,8.50
1,2024-10-17,NVIDIA CAFE SANTA CLARA CA,2.50
2,2024-10-17,NVIDIA CAFE SANTA CLARA CA,2.50
3,2024-10-17,NVIDIA CAFE SANTA CLARA CA,8.50
5,2024-10-21,SQ *ZAREEN'S MOUNTAIN gosq.com CA,22.21
...,...,...,...
97,2024-11-29,NVIDIA CAFE SANTA CLARA CA,8.50
98,2024-11-29,SATURA PATISSERIE LOS ALTOS CA,11.50
99,2024-11-29,SATURA PATISSERIE LOS ALTOS CA,24.25
100,2024-11-29,NVIDIA CAFE SANTA CLARA CA,2.50


Remove overlapping data that has already been added to the sheet:

In [5]:
latest_date = sheet.get_latest_col_value(col_mappings.get("date"))
new_data["datetime"] = pd.to_datetime(new_data["Date"])
new_data = new_data[new_data["Date"] >= latest_date]
display(new_data)

,Date,Name,Amount,datetime
95,2024-11-29,NVIDIA CAFE SANTA CLARA CA,0.55,2024-11-29
96,2024-11-29,NVIDIA CAFE SANTA CLARA CA,2.50,2024-11-29
97,2024-11-29,NVIDIA CAFE SANTA CLARA CA,8.50,2024-11-29
98,2024-11-29,SATURA PATISSERIE LOS ALTOS CA,11.50,2024-11-29
99,2024-11-29,SATURA PATISSERIE LOS ALTOS CA,24.25,2024-11-29
100,2024-11-29,NVIDIA CAFE SANTA CLARA CA,2.50,2024-11-29
101,2024-11-29,VOYAGER CAFE SANTA CLARA CA,7.50,2024-11-29


In [6]:
threshold = 0.85
new_data_labeled, low_conf = embed_and_classify(new_data, old_data, "description", "category" threshold)

KeyError: <src.spreadsheet.SpreadSheet object at 0x133b69d10>

In [ ]:
for index, name in low_conf.items():
    print(f"Item '{name}' at index {index} has low confidence and needs manual classification.")